---

# 🔧 TypeAdapter Validation

### 🎯 Intent

Validate **standalone Python types** (lists, dicts, tuples, etc.) without creating a `BaseModel`.

---

### 🧩 Core Components

1. **📝 Create TypeAdapter**

   * Define a `TypeAdapter` for any Python type.
   * Example: list, dict, tuple, union types.

2. **⚡ Methods**

   * `.validate_python(obj)` → validate Python objects.
   * `.validate_json(str)` → validate JSON input.

3. **📦 Coercion**

   * Converts compatible values automatically.
   * Example: `["1", 2]` → `[1, 2]`.

4. **🛑 Error Handling**

   * Raises `ValidationError` if input doesn’t match type.

5. **🌍 Use Cases**

   * Quick validation for configs, API payloads, ETL pipelines, etc.
   * Avoids writing a full model when not needed.

---


In [1]:
from pydantic import TypeAdapter, ValidationError
from typing import List, Dict, Union
import json

# 1. 📝 Simple list of integers
int_list_adapter = TypeAdapter(List[int])

print(int_list_adapter.validate_python(["1", 2, 3]))  
# ✅ Coerces "1" → 1 → [1, 2, 3]

try:
    int_list_adapter.validate_python(["a", 2, 3])  # invalid
except ValidationError as e:
    print("❌ Error (list[int]):", e.errors())


# 2. 📝 Dict with string keys and float values
dict_adapter = TypeAdapter(Dict[str, float])

print(dict_adapter.validate_python({"a": "1.5", "b": 2}))  
# ✅ Coerces "1.5" → 1.5


# 3. 📝 Union type: str | int
union_adapter = TypeAdapter(Union[int, str])

print(union_adapter.validate_python("123"))   # ✅ → "123" (string is valid)
print(union_adapter.validate_python(456))     # ✅ → 456


# 4. 📝 JSON validation
json_adapter = TypeAdapter(List[int])

print(json_adapter.validate_json("[1, 2, 3]"))  
# ✅ Valid JSON string → [1, 2, 3]

try:
    json_adapter.validate_json('["a", "b"]')  # invalid
except ValidationError as e:
    print("❌ Error (from JSON):", e.errors())


# 5. 🌍 Real-world: quick config validation
ConfigAdapter = TypeAdapter(Dict[str, Union[int, str]])

valid_cfg = ConfigAdapter.validate_python({"host": "localhost", "port": "8000"})
print("Config validated:", valid_cfg)


[1, 2, 3]
❌ Error (list[int]): [{'type': 'int_parsing', 'loc': (0,), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'a', 'url': 'https://errors.pydantic.dev/2.11/v/int_parsing'}]
{'a': 1.5, 'b': 2.0}
123
456
[1, 2, 3]
❌ Error (from JSON): [{'type': 'int_parsing', 'loc': (0,), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'a', 'url': 'https://errors.pydantic.dev/2.11/v/int_parsing'}, {'type': 'int_parsing', 'loc': (1,), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'b', 'url': 'https://errors.pydantic.dev/2.11/v/int_parsing'}]
Config validated: {'host': 'localhost', 'port': '8000'}
